# Problem Setting up
##### Notice: the example comes from wiki

In [0]:
states=('Healthy','Fever')
end_state='E'
observations=('normal','cold','dizzy')
start_probability={'Healthy':0.6,'Fever':0.4}
transition_probability={'Healthy':{'Healthy':0.69,'Fever':0.3,'E':0.01},
                        'Fever':{'Healthy':0.4,'Fever':0.59,'E':0.01}}
emission_probability={'Healthy':{'normal':0.5,'cold':0.4,'dizzy':0.1},
                      'Fever':{'normal':0.1,'cold':0.3,'dizzy':0.6}}

In [9]:
for i, observation_i_plus in enumerate(reversed(observations[1:]+(None,))):
  print(str(i)+':'+str(observation_i_plus))

0:None
1:dizzy
2:cold


In [0]:
def fwd_bwk(observations,states,start_proba,trans_proba,emm_proba,end_st): # the final state is not in the observation list
  ''' Forward process'''
  fwd=[] # a list to store a dictionary that the state distribution at each time step
  f_prev={}
  for i,observation_i in enumerate(observations):
    f_curr={} # a dict to store the probabilities of the states currently
    for st in states:
      if i==0:  # if it is the first time step the probability of the state is the initial probability
        prev_f_sum=start_proba[st]
      else: 
        '''if it is not the first time step, the probability of the state is the sum of 
        the all state transfromed from the previously time step given the current observation'''
        prev_f_sum=sum(f_prev[k]*trans_proba[k][st] for k in states)

      f_curr[st]=emm_proba[st][observation_i]*prev_f_sum

    fwd.append(f_curr)
    f_prev=f_curr
  p_fwd=sum(f_curr[k]*trans_proba[k][end_st] for k in states)

  '''Backward process'''
  bwk=[]
  p_prev={} # to store the state probabilities in the previous time step
  for i,observation_i_plus in enumerate(reversed(observations[1:]+(None,))):
    b_curr={}
    for st in states:
      if i==0: # the base case for backward process
        b_curr[st]=trans_proba[st][end_st]
      else:
        b_curr[st]=sum(trans_proba[st][l]*b_prev[l]*emm_proba[l][observation_i_plus] for l in states)
    bwk.insert(0,b_curr)
    b_prev=b_curr
  
  p_bwk=sum(start_proba[l]*emm_proba[l][observations[0]]*b_curr[l] for l in states)

  # Merging the two parts
  posterior=[]
  for i in range(len(observations)):
    posterior.append({st:fwd[i][st]*bwk[i][st]/p_fwd for st in states})

  # assert p_fwd==p_bwk
  return fwd,bwk,posterior,p_fwd


fwd,bwk,posterior,p_fwd=fwd_bwk(observations,states,start_probability,transition_probability,emission_probability,end_state)

In [19]:
fwd

[{'Fever': 0.04000000000000001, 'Healthy': 0.3},
 {'Fever': 0.03408, 'Healthy': 0.0892},
 {'Fever': 0.028120319999999997, 'Healthy': 0.007518}]

In [20]:
bwk

[{'Fever': 0.00109578, 'Healthy': 0.00104184},
 {'Fever': 0.00394, 'Healthy': 0.00249},
 {'Fever': 0.01, 'Healthy': 0.01}]

In [21]:
posterior

[{'Fever': 0.1229889624426741, 'Healthy': 0.8770110375573261},
 {'Fever': 0.3767719690490461, 'Healthy': 0.623228030950954},
 {'Fever': 0.7890472951586943, 'Healthy': 0.2109527048413057}]

In [27]:
p_fwd

0.00035638319999999995